# Face Mask Detection Project
This notebook will guide you through training a face mask detection model.

In [1]:
import cv2
print(cv2.__version__)

4.11.0


In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [3]:
image_folder = '/users/zarkhanhilal/Desktop/Face-Mask-Detection/dataset'
labels_list = ['with_mask', 'without_mask']
dataset = []
image_size = 100

In [4]:
for label in labels_list:
    folder_path = os.path.join(image_folder, label)
    label_index = labels_list.index(label)

    for file in tqdm(os.listdir(folder_path)):
        try:
            img_gray = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
            img_resized = cv2.resize(img_gray, (image_size, image_size))
            dataset.append([img_resized, label_index])
        except Exception as error:
            pass

100%|█████████████████████████████████████| 3828/3828 [00:01<00:00, 3526.66it/s]


In [5]:
features = []
labels = []

for img, lbl in dataset:
    features.append(img)
    labels.append(lbl)

In [6]:
features = np.array(features).reshape(-1, image_size, image_size, 1) / 255.0
labels = to_categorical(labels)

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
189/189 [==============================] - 9s 47ms/step - loss: 0.5871 - accuracy: 0.6955 - val_loss: 0.4592 - val_accuracy: 0.7876
Epoch 2/5
189/189 [==============================] - 9s 46ms/step - loss: 0.3870 - accuracy: 0.8290 - val_loss: 0.3785 - val_accuracy: 0.8332
Epoch 3/5
189/189 [==============================] - 9s 46ms/step - loss: 0.2818 - accuracy: 0.8830 - val_loss: 0.3158 - val_accuracy: 0.8564
Epoch 4/5
189/189 [==============================] - 9s 46ms/step - loss: 0.2135 - accuracy: 0.9134 - val_loss: 0.2884 - val_accuracy: 0.8868
Epoch 5/5
189/189 [==============================] - 9s 46ms/step - loss: 0.1581 - accuracy: 0.9369 - val_loss: 0.2899 - val_accuracy: 0.8815


In [9]:
model.save("mask_detector.h5")

/Users/zarkhanhilal/Desktop/Face-Mask-Detection/venv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
